In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']  = "0"
os.environ['CUDA_VISIBLE_DEVICES'] 

'0'

In [4]:
! pip install transformers
! pip install nlp

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.7 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 26.3 MB/s eta 0:00:01
     |████████████████████████████████| 21.9 MB 37.8 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
!rm -rf Rest/
!git clone https://github.com/alessandrocuda/Rest

Cloning into 'Rest'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 470 (delta 187), reused 285 (delta 101), pack-reused 87
Receiving objects: 100% (470/470), 160.48 MiB | 24.74 MiB/s, done.
Resolving deltas: 100% (195/195), done.
Checking out files: 100% (124/124), done.


In [2]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from transformers import TFBertForSequenceClassification
from nlp import load_dataset
import torch
import numpy as np
import pandas as pd
import json
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import ast

In [3]:
import sys
#root_project = "/content/Rest/"
root_project = "/storagenfs/a.cudazzo1/ReSt/"
#root_project = "/Users/Alessandro/Dev/repos/SaRaH/"
#root_project = "/home/jupyter/SaRaH/"
sys.path.append(root_project)
from src.data.utils import load_data, set_unkmark_token
from src.utils import metrics
from src.data.word_embedding import get_index_key_association, get_int_seq, build_keras_embedding_matrix, get_data_to_emb
from src.utils.utils import plotHistory, plotF1_macro, plotLoss
from src.utils.metrics import f1_macro



%load_ext autoreload
%autoreload 2

In [ ]:
!rm -rf /content/logs
!rm -rf /content/results

In [ ]:
def model_init():
    return BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path = "dbmdz/bert-base-italian-uncased")

In [4]:
model_name = "dbmdz/bert-base-italian-uncased"
#model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

In [ ]:
dataset_dev_path   = root_project + "dataset/haspeede2/preprocessed/dev/dev.csv"
dataset_test_tweets_path   = root_project + "dataset/haspeede2/preprocessed/reference/reference_tweets.csv"

df = pd.read_csv(dataset_dev_path, sep='\t')
df = df[['tokens','stereotype']]

df_test = pd.read_csv(dataset_test_tweets_path, sep='\t')
df_test = df_test[['tokens','stereotype']]

In [ ]:
df

In [ ]:
from pathlib import Path

def read_evalita_split(df):
    texts = []
    labels = []
    for index, row in df.iterrows():
      texts.append(row['tokens'])
      labels.append(row['stereotype'])
    return texts, labels

train_texts, train_labels = read_evalita_split(df)
test_texts, test_labels = read_evalita_split(df_test)

In [ ]:
#sample = ' '.join(ast.literal_eval(df[0]))
sample = train_texts[0]
print(sample)
input_ids = torch.tensor(tokenizer.encode(sample)).unsqueeze(0)  
token_list = tokenizer.convert_ids_to_tokens(tokenizer.encode(sample)) 
print(input_ids)
print(token_list)

In [9]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [10]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length = 50)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length = 50)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length = 50)

In [ ]:
import torch

class EvalitaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EvalitaDataset(train_encodings, train_labels)
val_dataset = EvalitaDataset(val_encodings, val_labels)
test_dataset = EvalitaDataset(test_encodings, test_labels)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/model7',          # output directory
    #learning_rate=1e-3,              # learning rate (default)
    num_train_epochs=3,              # total number of training epochs (3)
    per_device_train_batch_size=16,  # batch size per device during training (14)
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/logs/model7',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    eval_steps=100,
    evaluation_strategy="steps",
    load_best_model_at_end= True,
)

trainer7 = Trainer(
    #model=model,                         # the instantiated 🤗 Transformers model to be trained
    model_init=model_init,
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/dbmdz/bert-base-italian-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/64c19136528304a0aa80ab527d385022b5dfda00b76892d3577c29cf64eb7fc0.6ec690b98e01c56d26601258d2be34c3e5a76b949465ed58983cff81e5f9fa88
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layer

In [ ]:
trainer7.train()

In [ ]:
trainer7.evaluate()

In [ ]:
trainer7.predict(train_dataset)

In [ ]:
trainer7.predict(test_dataset)

---------------------------------------------------

Vedi -> https://huggingface.co/transformers/training.**html**

In [5]:
! pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-1.8.0-py3-none-any.whl (237 kB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached fsspec-2021.6.1-py3-none-any.whl (115 kB)
  Using cached multiprocess-0.70.12.2-py38-none-any.whl (128 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
from datasets import load_dataset, Dataset

In [6]:
dataset_dev_path   = root_project + "dataset/haspeede2/preprocessed/dev/dev.csv"
dataset_test_tweets_path   = root_project + "dataset/haspeede2/preprocessed/reference/reference_tweets.csv"

In [7]:
df = load_dataset('csv', data_files=dataset_dev_path, delimiter='\t', split='train')

Using custom data configuration default-0b80b7c4c8acb813
Reusing dataset csv (/storagenfs/a.cudazzo1/.cache/huggingface/datasets/csv/default-0b80b7c4c8acb813/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [8]:
df_splitted = df.train_test_split(test_size=0.1)

Loading cached split indices for dataset at /storagenfs/a.cudazzo1/.cache/huggingface/datasets/csv/default-0b80b7c4c8acb813/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-6b7ea8dce0c650b5.arrow and /storagenfs/a.cudazzo1/.cache/huggingface/datasets/csv/default-0b80b7c4c8acb813/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-a3ff1c3dfb6d13e0.arrow


In [9]:
df_splitted

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'hs', 'stereotype', 'processed_text', 'text_length', 'hashtags', '%CAPS-LOCK words', 'esclamations', 'questions', 'tokens', 'lemma', 'pos', 'dep', 'word_polarity', 'sentence_positive', 'sentence_negative', 'sentence_neutral', 'stem', '%bad_words'],
        num_rows: 6155
    })
    test: Dataset({
        features: ['id', 'text', 'hs', 'stereotype', 'processed_text', 'text_length', 'hashtags', '%CAPS-LOCK words', 'esclamations', 'questions', 'tokens', 'lemma', 'pos', 'dep', 'word_polarity', 'sentence_positive', 'sentence_negative', 'sentence_neutral', 'stem', '%bad_words'],
        num_rows: 684
    })
})

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding=True, truncation=True, max_length = 50)

In [11]:
tokenized_df = df_splitted.map(tokenize_function, batched=True)

Loading cached processed dataset at /storagenfs/a.cudazzo1/.cache/huggingface/datasets/csv/default-0b80b7c4c8acb813/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-78246de8bf19f5e3.arrow
Loading cached processed dataset at /storagenfs/a.cudazzo1/.cache/huggingface/datasets/csv/default-0b80b7c4c8acb813/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-e8908e69bb126be2.arrow


In [12]:
train_set = tokenized_df['train']
val_set = tokenized_df['test']

In [13]:
tf_train_dataset = train_set.with_format("tensorflow")
tf_eval_dataset = val_set.with_format("tensorflow")

In [14]:
train_features = {x: tf_train_dataset[x].to_tensor() for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["stereotype"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x].to_tensor() for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["stereotype"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [15]:
model = TFBertForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /usr/lib/python3.8/dist-packages/transformers/models/bert/modeling_tf_bert.py:1355 call  *
        outputs = self.bert(
    /usr/lib/python3.8/dist-packages/transformers/models/bert/modeling_tf_bert.py:614 call  *
        raise ValueError("You have to specify either input_ids or inputs_embeds")

    ValueError: You have to specify either input_ids or inputs_embeds
